In [ ]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import findspark

# Set Spark environment variables
os.environ["SPARK_HOME"] = "/home/iiitdmk-sic40/spark-3.5.5"  # Update if different
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"

# Initialize findspark
findspark.init()

from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder \
    .master("spark://172.16.72.48:7077") \
    .appName("Amazon_Reviews") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .getOrCreate()

print("Amazon_Reviews Spark Session Started 🚀")

Amazon_Reviews Spark Session Started 🚀


25/04/04 16:43:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
#Required Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [ ]:
df = spark.read.csv("hdfs://localhost:9000/user/iiitdmk/dataset/Amazon_reviews.csv", header=True, inferSchema=True)
df.show(5)  # Display the first 5 rows

+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
| Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|
+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
|  1|B001E4KFG0|A3SGXH7AUHU8GW|          delmartian|                   1|                     1|    5|1303862400|Good Quality Dog ...|I have bought sev...|
|  2|B00813GRG4|A1D87F6ZCVE5NK|              dll pa|                   0|                     0|    1|1346976000|   Not as Advertised|"Product arrived ...|
|  3|B000LQOCH0| ABXLMWJIXXAIN|"Natalia Corres "...|                   1|                     1|    4|1219017600|"""Delight"" says...|"This is a confec...|
|  4|B000UA0QIQ|A395BORC6FGVXV|                Karl|            

In [ ]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /home/iiitdmk-
[nltk_data]     sic27/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/iiitdmk-
[nltk_data]     sic27/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/iiitdmk-
[nltk_data]     sic27/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# ✅ Set Up PySpark for Jupyter Notebook
os.environ["SPARK_HOME"] = "/opt/spark"
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

In [ ]:
# ✅ Start Spark Session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Amazon_Reviews") \
    .getOrCreate()

25/04/04 16:44:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
# ✅ Load Data from HDFS
df = spark.read.csv("hdfs://localhost:9000/user/iiitdmk/dataset/Amazon_reviews.csv", header=True, inferSchema=True)
df

DataFrame[Id: int, ProductId: string, UserId: string, ProfileName: string, HelpfulnessNumerator: string, HelpfulnessDenominator: string, Score: string, Time: string, Summary: string, Text: string]

In [ ]:
# ✅ Convert only if df is a PySpark DataFrame
if isinstance(df, pyspark.sql.dataframe.DataFrame):
    df = df.toPandas()

print(df.head())  # Check if data is loaded correctly


   Id   ProductId          UserId                          ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                           delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                               dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  "Natalia Corres ""Natalia Corres"""   
3   4  B000UA0QIQ  A395BORC6FGVXV                                 Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    "Michael D. Bigham ""M. Wassir"""   

  HelpfulnessNumerator HelpfulnessDenominator Score        Time  \
0                    1                      1     5  1303862400   
1                    0                      0     1  1346976000   
2                    1                      1     4  1219017600   
3                    3                      3     2  1307923200   
4                    0                      0     5  1350777600   

                     Summary  \
0      Good Quality Dog Food   
1          Not as Advertised   
2  """Delight"" says it all"   
3             Cough Medi

In [ ]:
df

DataFrame[Id: int, ProductId: string, UserId: string, ProfileName: string, HelpfulnessNumerator: string, HelpfulnessDenominator: string, Score: string, Time: string, Summary: string, Text: string]

In [ ]:
# Convert to Pandas
df_pandas = df.toPandas()  # Convert entire DataFrame to Pandas

# ✅ Train-Test Split
x_train, x_test, y_train, y_test = train_test_split(df_pandas['Text'], df_pandas['Score'], test_size=0.3, random_state=0)


In [ ]:
# ✅ Text Preprocessing
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    soup = BeautifulSoup(text, "html.parser")
    review = soup.get_text()
    review = re.sub(r'[^a-zA-Z]', ' ', review).lower().split()
    review = [stemmer.stem(lemmatizer.lemmatize(word)) for word in review if word not in stop_words]
    return ' '.join(review)

corpus_train = [preprocess_text(text) for text in x_train]
corpus_test = [preprocess_text(text) for text in x_test]

[nltk_data] Downloading package wordnet to /home/iiitdmk-
[nltk_data]     sic27/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/iiitdmk-
[nltk_data]     sic27/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/tmp/ipykernel_26027/3780152681.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
/tmp/ipykernel_26027/3780152681.py:10: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# ✅ Define Text Column
text_column = "Text"  # Make sure this column exists in your DataFrame

# ✅ Ensure Data Exists
if text_column not in df.columns:
    raise ValueError(f"❌ ERROR: Column '{text_column}' not found in DataFrame!")

# ✅ Fill missing text values
df[text_column] = df[text_column].fillna("")

# ✅ Split Data into Train & Test
corpus_train, corpus_test = train_test_split(df[text_column], test_size=0.2, random_state=42)

# ✅ Apply TF-IDF Vectorization
tfidf_vec = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)
tfidf_vec_train = tfidf_vec.fit_transform(corpus_train)
tfidf_vec_test = tfidf_vec.transform(corpus_test)

print("✅ TF-IDF transformation completed successfully!")


✅ TF-IDF transformation completed successfully!


In [ ]:
# ✅ Logistic Regression (handles large sparse data better)
log_reg = LogisticRegression(max_iter=5000, solver="lbfgs")
log_reg.fit(tfidf_vec_train, y_train)

# ✅ Predictions
predictions = log_reg.predict(tfidf_vec_test)

# ✅ Accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"🎯 Logistic Regression Accuracy: {accuracy:.4f}")


🎯 Logistic Regression Accuracy: 0.7419
